# Crawling Subtitles of HK Movies

## Data Source

Subtitles are crawled from http://assrt.net/. 

Note: APIs are provided for personal use. We need to contact admin@assrt.net for commercial use. 

API details: http://assrt.net/api/doc#subs
+ It provides APIs for：
   + searching using keywords (API 1),
   + requesting details using movie id (API 2), 
   + and searching similar movie subtitles using movie id (API 3).  
   Note: only API 1 and API 2 are used in our implementation
+ The number of requests is limited to 20 per minute. 

In [1]:
# test APIs
import requests

# A token is required to request APIs
# Get your own token from http://assrt.net/usercp.php after registering
with open('./token.txt', 'r', encoding='utf-8') as f:
    token=f.readline()

In [2]:
# test "search" API (API 1)
params = {'token': token, 'q': '重慶森林', 'pos': 0, 'cnt': 15}
r = requests.get('http://api.assrt.net/v1/sub/search',params=params)

In [3]:
r.json()

{'sub': {'action': 'search',
  'keyword': '重慶森林',
  'result': 'succeed',
  'subs': [{'subtype': 'Subrip(srt)',
    'vote_score': 0,
    'videoname': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF',
    'revision': 0,
    'lang': {'desc': '繁', 'langlist': {'langcht': True}},
    'native_name': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF/[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF/[重慶森林].Chung.King.Express.1994.BDRip',
    'upload_time': '2009-12-10 10:26:49',
    'id': 135269},
   {'subtype': 'Subrip(srt)',
    'vote_score': 0,
    'videoname': '重庆森林 REMASTERED.EDiTiON.DVDRip.X264.DTS.iNT-ZY',
    'revision': 0,
    'lang': {'desc': '英', 'langlist': {'langeng': True}},
    'native_name': 'Chung.King.Express/重慶森林',
    'upload_time': '2007-07-13 00:02:00',
    'id': 64648}]},
 'status': 0}

In [4]:
# test "detail" API (API 2)
params = {'token': token, 'id': 135269}
detail = requests.get('http://api.assrt.net/v1/sub/detail',params=params)

In [5]:
detail.json()

{'sub': {'action': 'detail',
  'result': 'succeed',
  'subs': [{'view_count': 8344,
    'size': 20021,
    'lang': {'desc': '繁', 'langlist': {'langcht': True}},
    'filename': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF.rar',
    'subtype': 'Subrip(srt)',
    'vote_score': 0,
    'down_count': 255,
    'title': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF/\\\\[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF/\\\\\\\\[重慶森林].Chung.King.Express.1994.BDRip/\\\\\\\\\\\\[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF',
    'videoname': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF',
    'producer': {'uploader': 'happy777',
     'source': '',
     'producer': '',
     'verifier': 'happy777'},
    'url': 'http://file1.assrt.net/download/135269/%5B%E9%87%8D%E6%85%B6%E6%A3%AE%E6%9E%97%5D.Chung.King.Express.1994.BDRip.X264-TLF.rar?_=1636454137&-=37f6c481aa3097e9e4e8e965c6a75f48&api=1',
    'filelist': [{'s': '44KB',
      'f': '[重慶森林].Chung.King.Express.1994.BDRip.X264-TLF.srt',
      'url': 'ht

## How to obtain subtitles for massive movies?

We consider two approaches.   
One is to obtain a name list for HK Movies first. Then we use the name list to request the movie id using API 1 and use the returned movie id to request subtitle details (including download url). Finally we use the url to download the subtitle files. One limitation for this approach is that the selected name list may not be compact.  
Another is to go through all movies using all possible movie ids (6 digits, i.e., 0-999999) and identify which ones are in traditional Chinese. One limitation for this approach is the computational waste [1000000/20 = 50000 min = 833 hours are required to complete all requests due to the API request limit].

We utilized the first approach in our implementation.  

Details:

The name list for HK Movies (from Wikipedia)：
+ a whole list regardless of years: [List 1]  
https://zh.wikipedia.org/zh-hk/%E9%A6%99%E6%B8%AF%E9%9B%BB%E5%BD%B1%E5%88%97%E8%A1%A8
+ a list of HK Movies in decades: [List 2]  
https://zh.wikipedia.org/zh-hk/Category:%E5%90%84%E5%B9%B4%E4%BB%A3%E9%A6%99%E6%B8%AF%E9%9B%BB%E5%BD%B1%E4%BD%9C%E5%93%81 
+ a list of HK Movies in various years: [List 3] (not adopted)  
https://zh.wikipedia.org/zh-hk/Category:%E5%90%84%E5%B9%B4%E9%A6%99%E6%B8%AF%E9%9B%BB%E5%BD%B1%E5%88%97%E8%A1%A8 

The lists can be download from: 
+ List 1: https://docs.google.com/spreadsheets/d/105ftqNRja73EFJOr-PcMmM8a2JBfiEhrJnWrcpL7IYI/edit?usp=sharing
+ List 2: https://docs.google.com/spreadsheets/d/1uAKew08mYmML4S05cRWRm7goLS8_70WatYgGOIt8dfI/edit?usp=sharing

In [6]:
!pip install pysrt

In [7]:
import os
import json
import time
import logging
import requests
import openpyxl

from datetime import datetime
from collections import defaultdict

In [8]:
def get_logger(file_dir, file_name=None, level='DEBUG'):
    if not os.path.exists(file_dir):
        os.makedirs(file_dir)
    if not file_name:
        file_name = datetime.now().strftime('%Y%m%d_%H%M')
    file_path = os.path.join(file_dir, file_name+'.log') 
    
    logger = logging.getLogger()
    logger.handlers.clear()  # clear existed handler to avoid repeating printout
    logger.setLevel(level)
    BASIC_FORMAT = "%(asctime)s:%(levelname)s:%(message)s"
    DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
    formatter = logging.Formatter(BASIC_FORMAT, DATE_FORMAT)
    
    chlr = logging.StreamHandler() # set the handler for console printing
    chlr.setFormatter(formatter)
    chlr.setLevel('INFO')  # default level of chlr is set by logger.setLevel(level)
    
    fhlr = logging.FileHandler(file_path) # set the handle for file printing
    fhlr.setFormatter(formatter)
    
    # add the defined two handlers to the logger
    logger.addHandler(chlr)
    logger.addHandler(fhlr)
    
    return logger

In [9]:
logger = get_logger('./logs')

# A token is required to request APIs
# Get your own token from http://assrt.net/usercp.php after registering
with open('./token.txt', 'r', encoding='utf-8') as f:
    token = f.readline()

In [10]:
def save_json(data, data_file, output_dir='output'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    data_path = os.path.join(output_dir, data_file)
    with open(data_path, 'w', encoding='utf-8') as f:
        json.dump(data, f)
        logger.info('Saved to %s' % (data_path))
        
def load_json(data_file, output_dir='output'):
    data_path = os.path.join(output_dir, data_file)
    if not os.path.exists(data_path):
        logger.info("The file %s does not exist!" % data_file)
        return {}
    with open(data_path, 'r', encoding='utf-8') as f:
        logger.info('Load json from %s' % (data_path))
        return json.load(f)

def request_movie_candidates(movie_name):
    params = {'token': token, 'pos': 0, 'cnt': 15}
    params['q'] = movie_name
    try:
        return requests.get('http://api.assrt.net/v1/sub/search',params=params)
    except Exception as e:
        logger.error(e)
        logger.warning('Request Failed! Request movie name: ', movie_name)
        return None
    
def request_movie_details(movie_id):
    params = {'token': token}
    params['id'] = movie_id
    try:
        return requests.get('http://api.assrt.net/v1/sub/detail',params=params)
    except Exception as e:
        logger.error(e)
        logger.warning('Request Failed! Request movie id: ', movie_id)
        return None

def process_movie_list(movie_list_file='MovieList.xlsx'):
    movie_list = []
    workbook = openpyxl.load_workbook(movie_list_file)
    worksheet = workbook['whole']
    for col in worksheet.columns:
        for cell in col:
            cell_str = str(cell.value)
            if '電影名稱' not in cell_str and '[編輯]' not in cell_str:
                movie_list.append(cell_str)
        logger.info('%d movie names are obtained' % len(movie_list))
        break  # only the first column (movie names) is used
    return movie_list

def process_movie_list_yearly(movie_list_file='MovieListYearly.xlsx'):
    movie_list = []
    workbook = openpyxl.load_workbook(movie_list_file)
    for worksheet in workbook:
        for col in worksheet.columns:
            for cell in col:
                cell_str = str(cell.value)
                if cell_str not in ['0.0','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0']:
                    movie_list.append(cell_str.split('(')[0].strip())
            logger.info('%d movie names are obtained' % len(movie_list))
            break  # only the first column (movie names) is used
    return movie_list

def process_srt(file_name, movie_id, processed_dir='./HKMovies/processed/whole'):
    output_file = os.path.join(processed_dir, str(movie_id)+'.txt')
    encoding = find_encoding(file_name)
    subs = pysrt.open(file_name, encoding=encoding)
    if len(subs) > 1:
        with open(output_file, 'a', encoding='utf-8') as f:
            for sub in subs:
                f.write(sub.text.replace('-',''))
        logger.info("Write subtitles %s to %s" % (file_name, output_file))

In [11]:
# Initialize all required dictionaries and lists
num_name_matched = 0
id_list = []  # the list of all movie ids that are matched
id_list_unique = []  # the list of movie ids that are uniquely matched
name_id_dict = defaultdict(list)  # {movie_name: [all matched movie_ids, corresonding movie names]}
name_id_dict_unique = defaultdict(list)  # {movie_name: [uniquely matched movie_ids, corresonding movie names]}
name_id_dict_nonuniq = defaultdict(list)  # {movie_name: [>1 matched movie_ids, corresonding movie names]}
not_matched_movies = []  # the list of movie names that are not matched
processed_movies = []

# Process the whole movie list
movie_list = process_movie_list()
movie_list += process_movie_list_yearly()

2021-11-09 18:35:53:INFO:388 movie names are obtained
2021-11-09 18:35:53:INFO:5 movie names are obtained
2021-11-09 18:35:53:INFO:204 movie names are obtained
2021-11-09 18:35:53:INFO:496 movie names are obtained
2021-11-09 18:35:53:INFO:840 movie names are obtained
2021-11-09 18:35:53:INFO:1041 movie names are obtained
2021-11-09 18:35:53:INFO:1149 movie names are obtained
2021-11-09 18:35:53:INFO:1187 movie names are obtained
2021-11-09 18:35:53:INFO:1197 movie names are obtained


In [12]:
movie_list = set(movie_list)
logger.info("%d movie names are obtailed in total (repeated names are removed)" % len(movie_list))

2021-11-09 18:35:56:INFO:1399 movie names are obtailed in total (repeated names are removed)


In [13]:
# Request movie ids for movies in the movie_list
for movie_name in movie_list:
    if movie_name not in processed_movies:
        # request movie candidates given a movie name
        while True:
            r = request_movie_candidates(movie_name)
            if r is None: continue
            try:
                res_movie_list = r.json()['sub']['subs']
                break
            except Exception as e:
                logger.error(e)
                if r.status_code == 509:
                    time.sleep(3)
                    continue
                else:
                    break

        # match the requested results with the given movie name and specific language
        num_final_matched_i = 0
        for res_movie_i in res_movie_list:
            if movie_name in res_movie_i['videoname'] or movie_name in res_movie_i['native_name']:
                logger.info('%s: [%s, %s]' % (movie_name, res_movie_i['videoname'], res_movie_i['native_name']))  # change to log
                num_name_matched += 1
                if 'lang' not in res_movie_i:
                    continue
                if '繁' in res_movie_i['lang']['desc']:
                    matched_id = res_movie_i['id']
                    id_list.append(matched_id)
                    name_id_dict[movie_name].append([matched_id, res_movie_i['videoname'], res_movie_i['native_name']])
                    num_final_matched_i += 1

        # print out
        if num_final_matched_i == 0:
            not_matched_movies.append(movie_name)
            logger.warning('No movies are matched for %s' % movie_name)
        elif num_final_matched_i == 1:
            id_list_unique.append(matched_id)
            name_id_dict_unique[movie_name] = name_id_dict[movie_name]
        else:
            name_id_dict_nonuniq[movie_name] = name_id_dict[movie_name]
            logger.warning('More than one movies are matched for %s' % movie_name)
            logger.warning('Matched movie names: %s'% str(name_id_dict[movie_name]))
        
        processed_movies.append(movie_name)
        time.sleep(3)  # query limit: 20 times per minute

logger.info("%d movies are matched" % len(name_id_dict))
logger.info("%d movies are matched uniquely" % len(name_id_dict_unique))

2021-11-09 11:39:57:WARNING:No movies are matched for 天水圍的日與夜
2021-11-09 11:40:00:WARNING:No movies are matched for 開心鬼放暑假
2021-11-09 11:40:03:INFO:見鬼2: [The Eye 2 TLF, 見鬼2]
2021-11-09 11:40:03:INFO:見鬼2: [見鬼2 51Mov, The Eye 2]
2021-11-09 11:40:03:WARNING:More than one movies are matched for 見鬼2
2021-11-09 11:40:03:WARNING:Matched movie names: [[10748, 'The Eye 2 TLF', '見鬼2'], [9108, '見鬼2 51Mov', 'The Eye 2']]
2021-11-09 11:40:07:INFO:烂滚夫斗烂滚妻: [烂滚夫斗烂滚妻.Mr.And.Mrs.Player.2013.BluRay 港版蓝光, 蓝光原盘字幕]
2021-11-09 11:40:10:WARNING:No movies are matched for 饺子
2021-11-09 11:40:13:WARNING:No movies are matched for 情迷大話王
2021-11-09 11:40:16:INFO:死亡遊戲: [インシテミル 7日間のデス・ゲーム The.Incite.Mill.7-Day.Death.Game.1080p.BluRay.x264.DTS-jimmy, 死神鬥室(港)/算計：7日死亡遊戲(台)]
2021-11-09 11:40:16:INFO:死亡遊戲: [算計：7日死亡遊戲 The.Incite.Mill.2010.JAP.DVDRip.x264.AC3-zdzdz, ]
2021-11-09 11:40:16:WARNING:More than one movies are matched for 死亡遊戲
2021-11-09 11:40:16:WARNING:Matched movie names: [[206860, 'インシテミル 7日間のデス・ゲーム The.Incit

2021-11-09 11:43:19:INFO:姨妈的后现代生活: [姨妈的后现代生活].The.Postmodern.Life.Of.My.Aunt.2007.CN.DVDRip.XviD-DSDIY DSDIY, The.Postmodern.Life.Of.My.Aunt.2007/姨妈的后现代生活]
2021-11-09 11:43:23:ERROR:'sub'
2021-11-09 11:43:26:ERROR:'sub'
2021-11-09 11:43:29:INFO:十二金牌: [The.Twelve.Gold.Medallions.1970.DVDRip.XviD.iNT-Xl, 十二金牌]
2021-11-09 11:43:29:WARNING:No movies are matched for 十二金牌
2021-11-09 11:43:32:WARNING:No movies are matched for 緣份 (電影)
2021-11-09 11:43:35:ERROR:'sub'
2021-11-09 11:43:38:ERROR:'sub'
2021-11-09 11:43:41:WARNING:No movies are matched for 開心樂園
2021-11-09 11:43:44:WARNING:No movies are matched for 不死情謎
2021-11-09 11:43:48:ERROR:'sub'
2021-11-09 11:43:51:ERROR:'sub'
2021-11-09 11:43:54:WARNING:No movies are matched for 富貴再三逼人
2021-11-09 11:43:57:INFO:長輩: [Parental Guidance 1080p-CHD, 家长指导/長輩有交代/PG老爺指引]
2021-11-09 11:43:57:INFO:長輩: [Parental Guidance 2012 DVDRip XviD juggs, 父母指南/長輩有交代/家長指導/PG老爷指引]
2021-11-09 11:43:57:INFO:長輩: [長輩有交代 Parental Guidance 2012 DVDRip XviD juggs, 長輩有交代/家長指導

2021-11-09 11:46:48:INFO:錦衣衛: [錦衣衛/14 Blades/锦衣卫 14.Blades.2010.BluRay.1080p.AVC.DTS-HD.MA.5.1-CHD, 14 Blades/錦衣衛]
2021-11-09 11:46:48:INFO:錦衣衛: [14 Blades 14_Blades_2010_CN_DVDRip_Xvid_XTM, 14 Blades/錦衣衛/14 Blades/錦衣衛/錦衣衛]
2021-11-09 11:46:48:INFO:錦衣衛: [Blades.2010.CN.DVDRip.Xvid-XTM, 锦衣卫/錦衣衛/14 Blades]
2021-11-09 11:46:48:INFO:錦衣衛: [錦衣衛/锦衣卫/14 Blades 2010.CN.DVDRip.Xvid-XTM, ]
2021-11-09 11:46:48:INFO:錦衣衛: [Blades.2010.CN.DVDRip.Xvid-XTM, 錦衣衛/锦衣卫/錦衣衛/锦衣卫 14 Blades/錦衣衛/錦衣衛/錦衣衛/锦衣卫 14 Blades]
2021-11-09 11:46:48:WARNING:More than one movies are matched for 錦衣衛
2021-11-09 11:46:48:WARNING:Matched movie names: [[622656, '14.Blades.2010.720p.BluRay.x264.DTS-WiKi', '锦衣卫/14 Blades/錦衣衛'], [143752, '錦衣衛', '14.Blades.2010.DVDRip.XviD.AC3-ViSiON/錦衣衛/錦衣衛/錦衣衛/錦衣衛/錦衣衛'], [143580, '錦衣衛/14 Blades/锦衣卫 14.Blades.2010.BluRay.1080p.AVC.DTS-HD.MA.5.1-CHD', '14 Blades/錦衣衛'], [141703, '14 Blades 14_Blades_2010_CN_DVDRip_Xvid_XTM', '14 Blades/錦衣衛/14 Blades/錦衣衛/錦衣衛'], [141619, 'Blades.2010.CN.DVDRip.Xvid-XTM

2021-11-09 11:49:43:INFO:梁山伯與祝英台: [The.Butterfly.Lovers.Leon.and.Jo.2003-ASE, The.Butterfly.Lovers.Leon.and.Jo/蝴蝶夢－梁山伯與祝英台/蝴蝶夢－梁山伯與祝英台]
2021-11-09 11:49:43:WARNING:More than one movies are matched for 梁山伯與祝英台
2021-11-09 11:49:43:WARNING:Matched movie names: [[602338, '梁山伯與祝英台 (1963)', '梁山伯與祝英台 (1963)/梁山伯與祝英台 (1963)'], [602336, '梁山伯與祝英台 (1963)', '梁山伯與祝英台 (1963)/梁山伯與祝英台 (1963)'], [7798, 'The.Butterfly.Lovers.Leon.and.Jo.2003-ASE', 'The.Butterfly.Lovers.Leon.and.Jo/蝴蝶夢－梁山伯與祝英台/蝴蝶夢－梁山伯與祝英台']]
2021-11-09 11:49:46:INFO:神鵰俠侶: [九一神鵰俠侶, ]
2021-11-09 11:49:46:WARNING:No movies are matched for 神鵰俠侶
2021-11-09 11:49:49:ERROR:'sub'
2021-11-09 11:49:52:ERROR:'sub'
2021-11-09 11:49:55:WARNING:No movies are matched for 爛賭英雄
2021-11-09 11:49:58:WARNING:No movies are matched for AV
2021-11-09 11:50:02:ERROR:'sub'
2021-11-09 11:50:05:ERROR:'sub'
2021-11-09 11:50:08:INFO:雙龍會: [雙龍會, Twins.Dragons 1992]
2021-11-09 11:50:11:WARNING:No movies are matched for 風雨故園
2021-11-09 11:50:14:ERROR:'sub'
2021-11-09 11:

2021-11-09 11:53:08:ERROR:'sub'
2021-11-09 11:53:11:ERROR:'sub'
2021-11-09 11:53:14:INFO:霹靂火: [BluRay.1080p.x265.10bit.2Audio.MNHD-FRDS, 霹靂火]
2021-11-09 11:53:14:INFO:霹靂火: [The French Connection法国贩毒网 YYddr, The French Connection/霹靂神探/密探霹靂火]
2021-11-09 11:53:14:WARNING:More than one movies are matched for 霹靂火
2021-11-09 11:53:14:WARNING:Matched movie names: [[629166, 'BluRay.1080p.x265.10bit.2Audio.MNHD-FRDS', '霹靂火'], [100459, 'The French Connection法国贩毒网 YYddr', 'The French Connection/霹靂神探/密探霹靂火']]
2021-11-09 11:53:17:INFO:童眼: [The Child s Eye 2010 720p BluRay x264 DTS-WiKi, 童眼 2D 国/粤双语]
2021-11-09 11:53:17:INFO:童眼: [The Child s Eye 2010 720p BluRay x264 DTS-WiKi, 童眼 2D 国/粤双语]
2021-11-09 11:53:17:INFO:童眼: [The Child s Eye 2010 720p BluRay x264 DTS-WiKi, 童眼 2D 国/粤双语]
2021-11-09 11:53:17:INFO:童眼: [The Child's Eye 2010 BluRay 720p DTS 2Audio x264-CHD, 童眼 2D]
2021-11-09 11:53:17:INFO:童眼: [童眼3D Childs Eye (2010), Childs Eye]
2021-11-09 11:53:17:INFO:童眼: [童眼, ]
2021-11-09 11:53:17:INFO:童眼: [C

...

2021-11-09 13:53:56:INFO:特殊身份: [Special ID 2013 1080p BluRay x264-WiKi, 特殊身份 甄子丹 景甜 安志杰 张涵予  国粤双语]
2021-11-09 13:53:56:INFO:特殊身份: [Special ID, 特殊身份/终极解码]
2021-11-09 13:53:56:INFO:特殊身份: [特殊身份 Special.ID.2013.720p.BluRay.x264-WiKi, ]
2021-11-09 13:53:56:INFO:特殊身份: [特殊身份 Special.ID.2013.720p.BluRay.x264-WiKi, Special ID]
2021-11-09 13:53:59:ERROR:'sub'
2021-11-09 13:54:02:ERROR:'sub'
2021-11-09 13:54:05:WARNING:No movies are matched for 運財五福星
2021-11-09 13:54:08:INFO:吉星高照2015: [Lucky.Star.2015.720p.BluRay.x264-WiKi, 吉星高照2015/Lucky Star 2015]
2021-11-09 13:54:11:ERROR:'sub'
2021-11-09 13:54:14:ERROR:'sub'
2021-11-09 13:54:17:INFO:跟蹤: [Stalker.S01.E07-E08.1080p.WEB-DL.DD5.1.H.264-BS, 跟蹤者]
2021-11-09 13:54:17:INFO:跟蹤: [Stalker.S01E06.Love.Is.a.Battlefield.1080p.WEB-DL.DD5.1.H.264-BS, 跟蹤者/Stalker S01.E06.WEB-DL]
2021-11-09 13:54:17:INFO:跟蹤: [Stalker.S01E05.The.Haunting.1080p.WEB-DL.DD5.1.H.264-BS, 跟蹤者.WEB-DL/Stalker S01.E05]
2021-11-09 13:54:17:INFO:跟蹤: [Stalker.S01E04.Phobia.1080p.WEB-DL.DD5

2021-11-09 13:56:12:WARNING:No movies are matched for 射鵰英雄傳第三集
2021-11-09 13:56:16:ERROR:'sub'
2021-11-09 13:56:19:ERROR:'sub'
2021-11-09 13:56:22:WARNING:No movies are matched for 我愛廚房
2021-11-09 13:56:25:INFO:蝴蝶: [不知道, 蝴蝶梦.Rebecca.1940]
2021-11-09 13:56:25:INFO:蝴蝶: [Butterfly's Tongue.1999.DVDRip.x264.蝴蝶的舌头, La lengua de las mariposas/ Butterfly's Tongue]
2021-11-09 13:56:25:INFO:蝴蝶: [Rebecca.2020.1080p.NF.WEB-DL.DDP5.1.Atmos.x264-MZABI, 蝴蝶梦/Rebecca]
2021-11-09 13:56:25:INFO:蝴蝶: [Rebecca.2020.1080p.NF.WEB-DL.DDP5.1.Atmos.x264-MZABI , 蝴蝶夢/《蝴蝶梦2020》| Netflix 官方网站]
2021-11-09 13:56:25:INFO:蝴蝶: [M.Butterfly.1993.1080p.WEBRip.x264-RARBG, 蝴蝶君]
2021-11-09 13:56:25:INFO:蝴蝶: [[蝴蝶梦].Rebecca.1940.CC.BluRay.1080p.x264.FLAC.3Audios-CMCT.mkv, 蝴蝶梦 Rebecca [蝴蝶梦].Rebecca.1940.CC.BluRay.1080p.x264.FLAC.3Audios-CMCT.mkv]
2021-11-09 13:56:25:INFO:蝴蝶: [M. Butterfly.1993.DVDRip.XVid, 蝴蝶君]
2021-11-09 13:56:25:INFO:蝴蝶: [M. Butterfly.1993.DVDRip.XVid, 蝴蝶君]
2021-11-09 13:56:25:INFO:蝴蝶: [Butterfly.S01.720p.HDT

2021-11-09 13:58:38:INFO:精武英雄: [精武英雄 [精武英雄].Fist.of.Legend.1994.BluRay.US.1080p.DD2.0.2Audio.x264-CHD, Fist of Legend/精武英雄/Fist of Legend/精武英雄/精武英雄/Fist of Legend]
2021-11-09 13:58:38:INFO:精武英雄: [精武英雄.Fist.of.Legend.1994, 精武英雄.Fist.of.Legend.1994]
2021-11-09 13:58:38:WARNING:No movies are matched for 精武英雄
2021-11-09 13:58:41:INFO:狼牙: [Legendary Assassin 2008 720p BluRay x264 DTS-WiKi, 狼牙/狼牙之阿布 ]
2021-11-09 13:58:41:WARNING:No movies are matched for 狼牙
2021-11-09 13:58:45:ERROR:'sub'
2021-11-09 13:58:48:ERROR:'sub'
2021-11-09 13:58:51:INFO:A計劃: [BluRay.1080p.x265.10bit.2Audio.MNHD-FRDS, A計劃續集/Project.A.II.1987.HEVC]
2021-11-09 13:58:54:INFO:婚前試愛: [婚前试爱 婚前試愛 Marriage.Wa.A.Liar.2010.BluRay.1080p.AC3.2Audio.x264-CHD 國粵 7.1G, Marriage.Wa.A.Liar/婚前試愛]
2021-11-09 13:58:54:WARNING:No movies are matched for 婚前試愛
2021-11-09 13:58:57:ERROR:'sub'
2021-11-09 13:59:00:ERROR:'sub'
2021-11-09 13:59:03:WARNING:No movies are matched for 鷹王
2021-11-09 13:59:06:WARNING:No movies are matched for 法網情
2021-1

2021-11-09 14:01:11:WARNING:More than one movies are matched for 彈道
2021-11-09 14:01:11:WARNING:Matched movie names: [[146586, 'Ballistic.2008.CN.DVDRip.Xvid-XTM', 'Ballistic/彈道/Ballistic/彈道/彈道'], [195922, '彈道 R3DVD', 'Ballistic']]
2021-11-09 14:01:14:ERROR:'sub'
2021-11-09 14:01:17:ERROR:'sub'
2021-11-09 14:01:20:INFO:六福喜事: [六福喜事.Hello Babies.2014.BDRip.X264.DualAudio.iNT.TLF, ]
2021-11-09 14:01:20:INFO:六福喜事: [六福喜事.Hello.Babies.2014.BluRay 港版蓝光, 蓝光原盘字幕]
2021-11-09 14:01:24:WARNING:No movies are matched for 我愛HK開心萬歲
2021-11-09 14:01:27:ERROR:'sub'
2021-11-09 14:01:30:ERROR:'sub'
2021-11-09 14:01:33:WARNING:No movies are matched for 好女十八嫁
2021-11-09 14:01:36:INFO:復仇者之死: [復仇者之死 Revenge.a.Love.Story.2010.BluRay.720p.AC3.2Audio.x264-CHD, Revenge : A Love Story/复仇者之死]
2021-11-09 14:01:39:ERROR:'sub'
2021-11-09 14:01:42:ERROR:'sub'
2021-11-09 14:01:45:WARNING:No movies are matched for 選老頂
2021-11-09 14:01:49:INFO:意外: [不知道, Trafic (1971)/聪明笨伯古惑车/交通意外]
2021-11-09 14:01:49:INFO:意外: [CatchPlay官方

2021-11-09 14:03:40:WARNING:No movies are matched for 龍拳
2021-11-09 14:03:43:ERROR:'sub'
2021-11-09 14:03:46:ERROR:'sub'
2021-11-09 14:03:50:INFO:紅玫瑰白玫瑰: [The Crimson Petal and the White, 絳紅雪白的花瓣/绛红雪白的花瓣/紅玫瑰白玫瑰/红玫瑰白玫瑰/深紅與白色的花瓣/深红与白色的花瓣]
2021-11-09 14:03:53:INFO:犀照: [49.Days.2006.DVDRip.XviD-ESPiSE /犀照, 49 Days]
2021-11-09 14:03:53:INFO:犀照: [犀照49.Days.2006.DVDSCR.XviD-SVO, 犀照]
2021-11-09 14:03:53:INFO:犀照: [49.Days.2006.DVDSCR.XviD-SVO.犀照, ]
2021-11-09 14:03:53:WARNING:More than one movies are matched for 犀照
2021-11-09 14:03:53:WARNING:Matched movie names: [[35668, '49.Days.2006.DVDRip.XviD-ESPiSE /犀照', '49 Days'], [35393, '犀照49.Days.2006.DVDSCR.XviD-SVO', '犀照']]
2021-11-09 14:03:56:ERROR:'sub'
2021-11-09 14:03:59:ERROR:'sub'
2021-11-09 14:04:02:WARNING:No movies are matched for 偶遇
A Sudden Love
2021-11-09 14:04:05:INFO:蜜桃成熟时33D: [蜜桃成熟时33D 蓝光版本, The 33D Invader]
2021-11-09 14:04:05:INFO:蜜桃成熟时33D: [【蜜桃成熟时33D】The 33D Invader/蓝光原盘字幕 Mysilu, ]
2021-11-09 14:04:05:INFO:蜜桃成熟时33D: [蜜桃成熟时33D.The

In [14]:
# Save updated dictionaries and lists to json files
save_json(name_id_dict, 'name_id_dict.json', 'output')
save_json(name_id_dict_unique, 'name_id_dict_unique.json', 'output')
save_json(name_id_dict_nonuniq, 'name_id_dict_nonuniq.json', 'output')
save_json(not_matched_movies, 'not_matched_movies.json', 'output')
save_json(id_list, 'id_list.json', 'output')
save_json(id_list_unique, 'id_list_unique.json', 'output')

2021-11-09 16:10:05:INFO:Saved to output/name_id_dict.json
2021-11-09 16:10:05:INFO:Saved to output/name_id_dict_unique.json
2021-11-09 16:10:05:INFO:Saved to output/name_id_dict_nonuniq.json
2021-11-09 16:10:05:INFO:Saved to output/not_matched_movies.json
2021-11-09 16:10:05:INFO:Saved to output/id_list.json
2021-11-09 16:10:05:INFO:Saved to output/id_list_unique.json


In [20]:
# Request download url for given movie ids and download data from the obtained url
id_url_dict = load_json('id_url_dict.json')
name_id_dict = load_json('name_id_dict.json')

download_dir = './download'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    
for movie_name, movie_id_name in name_id_dict.items():
    # take the first movie id if one movie has > 1 ids
    movie_id = movie_id_name[0][0]
    
    if movie_id not in id_url_dict:
        while True:
            r = request_movie_details(movie_id)
            if r is None: continue
            try:
                res_movie_list = r.json()['sub']['subs']
                break
            except Exception as e:
                logger.error(e)
                if r.status_code == 509:
                    time.sleep(3)
                    continue
                else:
                    break
        download_http = res_movie_list[0]['url'].split('?')[0].replace('file0.','').replace('file1.','')
        id_url_dict[movie_id] = download_http
    else:
        download_http = id_url_dict[movie_id]
    logger.info('download_http: %s' % download_http)
    suffix = download_http.split('.')[-1]
    output_name = str(movie_id)+'.'+suffix
    download_http = '"' + download_http + '"'
    !curl -L -o$download_dir/$output_name $download_http

if id_url_dict:
    save_json(id_url_dict, 'id_url_dict.json', 'output')

2021-11-09 18:41:36:INFO:Load json from output/id_url_dict.json
2021-11-09 18:41:36:INFO:Load json from output/name_id_dict.json
2021-11-09 18:41:36:INFO:download_http: http://assrt.net/download/010748/The.Eye.2.2004.DVDRip.XviD-TLF.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1007      0 --:--:-- --:--:-- --:--:--  1014
100 32587  100 32587    0     0   126k      0 --:--:-- --:--:-- --:--:--  430k


2021-11-09 18:41:36:INFO:download_http: http://assrt.net/download/258542/Mr.And.Mrs.Player.2013.BluRay.rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1642      0 --:--:-- --:--:-- --:--:--  1642
100 70658  100 70658    0     0  12161      0  0:00:05  0:00:05 --:--:-- 20061


2021-11-09 18:41:42:INFO:download_http: http://assrt.net/download/206860/The.Incite.Mill.7-Day.Death.Game.1080p.BluRay.x264.DTS-jimmy.R3TW.rar


...

2021-11-09 20:40:28:INFO:download_http: http://assrt.net/download/202002/%E3%80%90%E8%9C%9C%E6%A1%83%E6%88%90%E7%86%9F%E6%97%B633D%E3%80%91The%2033D%20Invader%E8%93%9D%E5%85%89%E5%8E%9F%E7%9B%98%E5%AD%97%E5%B9%95.rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1483      0 --:--:-- --:--:-- --:--:--  1483
100 59401  100 59401    0     0  10208      0  0:00:05  0:00:05 --:--:-- 13133


2021-11-09 20:40:34:INFO:download_http: http://assrt.net/download/643106/Terms_of_Endearment_(1983)_%E8%A6%AA%E5%AF%86%E9%97%9C%E4%BF%82.rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1468      0 --:--:-- --:--:-- --:--:--  1452
100  724k  100  724k    0     0   117k      0  0:00:06  0:00:06 --:--:--  187k


2021-11-09 20:40:40:INFO:Saved to output/id_url_dict.json


In [12]:
# redownload the failed files
download_dir = "./download"
movie_id_todo = []
for file_name in os.listdir(download_dir):
    file_path = os.path.join(download_dir, file_name)
    fsize = os.path.getsize(file_path)
    if fsize == 0:
        movie_id_todo.append(file_name.split('.')[0])
        print(file_name)

id_url_dict = load_json('id_url_dict.json')        
for movie_id in movie_id_todo:
    if movie_id not in id_url_dict:
#         continue
        while True:
            r = request_movie_details(movie_id)
            if r is None: continue
            try:
                res_movie_list = r.json()['sub']['subs']
                break
            except Exception as e:
                logger.error(e)
                if r.status_code == 509:
                    time.sleep(3)
                    continue
                else:
                    break
        download_http = res_movie_list[0]['url'].split('?')[0].replace('file0.','').replace('file1.','')
        id_url_dict[movie_id] = download_http
    else:
        download_http = id_url_dict[movie_id]
    logger.info('download_http: %s' % download_http)
    suffix = download_http.split('.')[-1]
    output_name = str(movie_id)+'.'+suffix
    download_http = '"' + download_http + '"'
    !curl -L -o$download_dir/$output_name $download_http

2021-11-10 10:31:42:INFO:Load json from output/id_url_dict.json
2021-11-10 10:31:42:INFO:download_http: http://assrt.net/download/257304/As.the.Light.Goes.Out.2014.BluRay.rar


257304.rar
605797.rar
211173.rar
16037.rar
227409.rar
644654.rar
643130.rar
623588.rar
644236.rar
670828.zip
4780.rar
644380.zip
94023.rar
645277.rar
643197.rar
238813.rar
12384.rar
250029.rar
146586.rar
143708.rar
178677.rar
242465.rar
5579.rar
625502.rar
621064.rar
620691.rar
204053.rar
229325.rar
57528.rar
97916.rar
621947.rar
77392.rar
192409.rar
234431.rar
6584.rar
629963.zip
626042.rar
201136.rar
34745.rar
103180.rar
111220.rar
180794.rar
186773.rar
627045.zip
604632.rar
641703.rar
629169.zip
240231.rar
128516.rar
139636.rar
108037.rar
85761.zip
17142.rar
96352.rar
629164.zip
629167.zip
175662.rar
626711.rar
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1254      0 --:--:-- --:--:-- --:--:--  1254
100 61975  100 61975    0     0   272k      0 --:--:-- --:--:-- --:--:--  272k


2021-11-10 10:31:43:INFO:download_http: http://assrt.net/download/605797/Barry%20Lyndon%20(1975).rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1483      0 --:--:-- --:--:-- --:--:--  1483
100 4644k  100 4644k    0     0   612k      0  0:00:07  0:00:07 --:--:-- 1412k


2021-11-10 10:31:50:INFO:download_http: http://assrt.net/download/211173/Peace%20Hotel%20(1995).rar


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0--:-- --:--:-- --:--:--     0


2021-11-10 10:35:32:INFO:download_http: http://assrt.net/download/175662/%E7%8E%89%E5%A5%B3%E6%B7%BB%E4%B8%81.rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


2021-11-10 10:35:32:INFO:download_http: http://assrt.net/download/626711/Iron.Fists.And.Kung-Fu.Kicks.2019.1080p.NF.WEBRip.DDP5.1.x264-pawel2006.rar


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [ ]:
for movie_id in movie_id_todo:
    while True:
        r = request_movie_details(movie_id)
        if r is None: continue
        try:
            res_movie_list = r.json()['sub']['subs']
            break
        except Exception as e:
            logger.error(e)
            if r.status_code == 509:
                time.sleep(3)
                continue
            else:
                break
    download_http = res_movie_list[0]['url'].split('?')[0].replace('file0.','').replace('file1.','')
    id_url_dict[movie_id] = download_http
    logger.info('download_http: %s' % download_http)
    suffix = download_http.split('.')[-1]
    output_name = str(movie_id)+'.'+suffix
    download_http = '"' + download_http + '"'
    !curl -L -o./HKMovies/download/yearly/$output_name $download_http

In [ ]:
# Decompress the download files
download_dir = './download'
decompre_dir = './decompressed'
if not os.path.exists(decompre_dir):
    os.makedirs(decompre_dir)

count = 0
for file_name in os.listdir(download_dir):
    download_path = os.path.join(download_dir, file_name)
    suffix = file_name.split('.')[-1]
    movie_id = file_name.split('.')[0]
    decompre_path = os.path.join(decompre_dir, movie_id)
    try:
        if suffix == 'zip':
            !unzip -o $download_path -d $decompre_path
        elif suffix == 'rar':
            decompre_path += '/'
            !unrar x -o+ $download_path $decompre_path
        count+=1
    except Exception as e:
        logger.warning(e)

In [ ]:
# Filter the download subtitles
filtered_path = './decompressed_filtered/'
for movie_id in os.listdir(filtered_path):
    movie_id_path = os.path.join(filtered_path, movie_id)
    for file_name in os.listdir(movie_id_path):
        file_path = os.path.join(movie_id_path, file_name)
        if os.path.isdir(file_path):
            logger.info(str(file_path))
            continue
        if file_name.endswith('chs.srt') or file_name.endswith('eng.srt') \
        or file_name.endswith('en.srt') or file_name.endswith('EN.srt') \
        or file_name.endswith('CHS.srt') or file_name.endswith('CS.srt') \
        or file_name.endswith('Eng.srt') or file_name.endswith('txt') \
        or file_name.endswith('jpg') or file_name.endswith('png'):
            file_path = '"' + file_path + '"'
            !rm $file_path
            logger.info("rm %s" % file_path)

In [ ]:
# Process the filtered subtitles
processed_dir = './processed'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

count = 0
for movie_id in os.listdir(filtered_path):
    movie_id_path = os.path.join(filtered_path, movie_id)
    for file_name in os.listdir(movie_id_path):
        file_path = os.path.join(movie_id_path, file_name)
        if os.path.isdir(file_path):
            logger.info(str(file_path))
        suffix = file_name.split('.')[-1]
        try:
            if suffix == 'srt':
                process_srt(file_path, movie_id)
#         elif suffix == 'ass':
        except Exception as e:
            logger.warning(e)
            logger.warning('Corresponding movie_id: %s' % str(movie_id))
            continue
            

In [ ]:
id_name_dict = {}
for key, value in name_id_dict.items():
    id_name_dict[str(value[0][0])] = key
save_json(id_name_dict, 'id_name_dict.json', 'output')

In [ ]:
for movie_id in os.listdir(filtered_path):
    print(movie_id, id_name_dict.get(str(movie_id)))

In [ ]:
# Count the number of tokens in the processed subtitles
from tokenizers.implementations import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing
tokenizer = BertWordPieceTokenizer("../NLP_models/CantoneseBERT/test/models/electra/vocab.txt")

num_tokens = 0
article_tokens_count = defaultdict(int)
i = 0
processed_dir = './processed'
for sub_file in os.listdir(processed_dir):
    if sub_file.endswith('txt'):
        sub_path = os.path.join(processed_dir, sub_file)
        with open(sub_path, "r", encoding="utf-8") as f:
            for line in f:
                num_tokens_i = len(tokenizer.encode(line).ids)
                num_tokens += num_tokens_i
                article_tokens_count[num_tokens_i] += 1
        i += 1
        if i%10 == 0:
            print("Counted %d articles: %d tokens till now" % (i, num_tokens))
print("Counted %d articles: %d tokens" % (i, num_tokens))